In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [11]:
# pre-process nyc mobility data

# load nyc data
nyc_mobility_2020 = pd.read_csv('/Users/luchen/Documents/MSUA/Capstone/datasets/Google Mobility Data/nyc_mobility_2020.csv', na_values='')
nyc_mobility_2021 = pd.read_csv('/Users/luchen/Documents/MSUA/Capstone/datasets/Google Mobility Data/nyc_mobility_2020.csv', na_values='')
nyc_mobility_2022 = pd.read_csv('/Users/luchen/Documents/MSUA/Capstone/datasets/Google Mobility Data/nyc_mobility_2020.csv', na_values='')

# merge nyc data
nyc_mobility = pd.concat([nyc_mobility_2020, nyc_mobility_2021, nyc_mobility_2022], ignore_index=True)
nyc_mobility = nyc_mobility.drop(['month', 'day'], axis=1)

print("Data shape:", nyc_mobility.shape)
print(nyc_mobility.head())

Data shape: (4815, 15)
  country_region_code country_region sub_region_1  sub_region_2  metro_area  \
0                  US  United States     New York  Bronx County         NaN   
1                  US  United States     New York  Bronx County         NaN   
2                  US  United States     New York  Bronx County         NaN   
3                  US  United States     New York  Bronx County         NaN   
4                  US  United States     New York  Bronx County         NaN   

   iso_3166_2_code  census_fips_code                     place_id        date  \
0              NaN           36005.0  ChIJBUEf6ovgwokRwlazSIxIpsk  2020-02-15   
1              NaN           36005.0  ChIJBUEf6ovgwokRwlazSIxIpsk  2020-02-16   
2              NaN           36005.0  ChIJBUEf6ovgwokRwlazSIxIpsk  2020-02-17   
3              NaN           36005.0  ChIJBUEf6ovgwokRwlazSIxIpsk  2020-02-18   
4              NaN           36005.0  ChIJBUEf6ovgwokRwlazSIxIpsk  2020-02-19   

   retail_and_r

In [ ]:
# filter nyc counties data
mobility_2020 = all_mobility_2020.dropna(subset=['sub_region_2'])